# Create Resources

# ! IMPORTANT
Change \<var\> with your variable

In [ ]:
import chi

chi.use_site("CHI@UC")
chi.set("project_name", "<insert projects code, such as CH-XXXXXX")
chi.set("image", "CC-CentOS7")
chi.set("keypair_name", "<insert keypair name>")

In [ ]:
from chi import lease

res = []
lease.add_node_reservation(res, node_type="compute_skylake", count=1)
lease.add_fip_reservation(res, count=1)
start_date, end_date = lease.lease_duration(days=7)

l = lease.create_lease("minio-k8", res, start_date=start_date, end_date=end_date)

print("Waiting for lease to start ...")

l = lease.wait_for_active(l["id"])

print("Lease started!")

reserved_fips = lease.get_reserved_floating_ips(l["id"])
print("ip =",reserved_fips[0])

In [ ]:
from chi import server
reservation_id = ""
for item in l["reservations"]:
    if item["resource_type"] == "physical:host":
        reservation_id = item["id"]

instance = server.create_server(
    "minio-k8-instance", 
    reservation_id=reservation_id, 
    image_name=chi.get("image")
)

print("Creating instance...")
chi.server.wait_for_active(instance.id, timeout=1200)

print("Associating floating ip")
ip = server.associate_floating_ip(instance.id, reserved_fips[0])

server.wait_for_tcp(ip, port=22)
print("Done! IP=", ip)

# ! IMPORTANT
If the code below fails, just re-run it again

In [ ]:
from chi.ssh import Remote

r = Remote(ip=ip)

In [ ]:
r.run('echo "Hello world!"')

# Setup loop devices

In [ ]:
r.run("sudo mkdir /mnt/extra")
r.run("sudo chown cc /mnt/extra")
r.run("mkdir -p /mnt/extra/loop-files/")

# Create loop devices
with r.cd("/mnt/extra/loop-files/"):
    r.run("sudo dd if=/dev/zero of=loopbackfile1.img bs=100M count=250")
    r.run("sudo dd if=/dev/zero of=loopbackfile2.img bs=100M count=250")
    r.run("sudo dd if=/dev/zero of=loopbackfile3.img bs=100M count=250")
    r.run("sudo dd if=/dev/zero of=loopbackfile4.img bs=100M count=250")
    r.run("sudo dd if=/dev/zero of=loopbackfile5.img bs=100M count=250")

    r.run("sudo losetup -fP loopbackfile1.img")
    r.run("sudo losetup -fP loopbackfile2.img")
    r.run("sudo losetup -fP loopbackfile3.img")
    r.run("sudo losetup -fP loopbackfile4.img")
    r.run("sudo losetup -fP loopbackfile5.img")
print("Created loop devices")

# Format loop devices
r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile1.img')
r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile2.img')
r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile3.img')
r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile4.img')
r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile5.img')
print("Formatted loop devices")

# Mount loop devices
r.run("mkdir -p /mnt/extra/loop-devs/loop0")
r.run("mkdir -p /mnt/extra/loop-devs/loop1")
r.run("mkdir -p /mnt/extra/loop-devs/loop2")
r.run("mkdir -p /mnt/extra/loop-devs/loop3")
r.run("mkdir -p /mnt/extra/loop-devs/loop4")

with r.cd("/mnt/extra/loop-files/"):
    r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop0")
    r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop1")
    r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop2")
    r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop3")
    r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop4")
print("Mounted loop devices")

# Deploy kubernetes using kubeadm

In [ ]:
# Install CRI-O
r.run("sudo curl -L -o /etc/yum.repos.d/devel:kubic:libcontainers:stable.repo https://download.opensuse.org/repositories/devel:/kubic:/libcontainers:/stable/CentOS_7/devel:kubic:libcontainers:stable.repo")
r.run("sudo curl -L -o /etc/yum.repos.d/devel:kubic:libcontainers:stable:cri-o:1.24.repo https://download.opensuse.org/repositories/devel:kubic:libcontainers:stable:cri-o:1.24/CentOS_7/devel:kubic:libcontainers:stable:cri-o:1.24.repo")
r.run("sudo yum install cri-o -y")
print("Installed Cri-o")

# Disable swap
r.run("sudo swapoff -a")
r.run("sudo sed -i '/swap/d' /etc/fstab")
print("swap disabled")

# Configure ufw config
r.run('cat <<EOF | sudo tee /usr/lib/sysctl.d/00-system.conf \nnet/bridge/bridge-nf-call-ip6tables = 1 \nnet/bridge/bridge-nf-call-iptables = 1 \nnet/bridge/bridge-nf-call-arptables = 1 \nnet/ipv4/ip_forward = 1 \nEOF')
print("UFW configured")

# Enable bridge and br_netfilter modprobe
r.run("sudo modprobe bridge")
r.run("sudo modprobe br_netfilter")
print("Modprobe enabled")

# Modify sysctl config
r.run("sudo ufw reload")
r.run("sudo sysctl --system")
print("Sysctl config modified")

# Add kubernetes repo
r.run('cat <<EOF | sudo tee /etc/yum.repos.d/kubernetes.repo \n[kubernetes] \nname=Kubernetes \nbaseurl=https://packages.cloud.google.com/yum/repos/kubernetes-el7-\$basearch \nenabled=1 \ngpgcheck=1 \nrepo_gpgcheck=0 \ngpgkey=https://packages.cloud.google.com/yum/doc/yum-key.gpg https://packages.cloud.google.com/yum/doc/rpm-package-key.gpg \nexclude=kubelet kubeadm kubectl \nEOF')
print("Kubernetes repo added")

# Set SELinux in permissive mode
# r.run("sudo setenforce 0")
r.run("sudo sed -i 's/^SELINUX=enforcing$/SELINUX=permissive/' /etc/selinux/config")
r.run("yum repolist -y")
print("Changed SELinux")

# Install kubernetes
r.run("sudo yum install -y kubelet-1.24.0 kubeadm-1.24.0 kubectl-1.24.0 --disableexcludes=kubernetes")
print("Kubernetes installed")

# Enable CRI-O and kubelet
r.run("sudo systemctl daemon-reload")
r.run("sudo systemctl enable crio --now")
r.run("sudo systemctl enable kubelet --now")
print("Cri-o and kubelet enabled")

# Initialize control plane
r.run("sudo kubeadm init --pod-network-cidr=192.168.0.0/16")
print("Control plane initialized")

# Add kubectl alias
r.run("mkdir -p ~/.kube")
r.run("sudo cp -i /etc/kubernetes/admin.conf ~/.kube/config")
r.run("sudo chown cc ~/.kube/config")
print("kubectl alias added")

# Apply Calico for networking
r.run("kubectl create -f https://projectcalico.docs.tigera.io/manifests/tigera-operator.yaml")
r.run("kubectl create -f https://projectcalico.docs.tigera.io/manifests/custom-resources.yaml")
print("Calico applied")

# Untaint master node
r.run("kubectl taint nodes --all node-role.kubernetes.io/control-plane- node-role.kubernetes.io/master-")
print("Master node untainted")

# Quickstart MinIO

# ! IMPORTANT
If r.put() fails, comment it and uncomment the r.run() command

In [ ]:
# Download custom minio-dev yaml. It contains default
# r.run("wget https://raw.githubusercontent.com/dParikesit/tutorial-collections/main/quickstart-minio-kubernetes/minio-dev.yaml")
r.put("./minio-dev.yaml")
print("Config downloaded")

In [ ]:
# Create MinIO pods
r.run("kubectl apply -f ~/minio-dev.yaml")
print("MinIO pods created")

# Use the API

Open 2 terminals

In the first terminal, run   
```kubectl port-forward pod/minio -n minio-dev 9000 9090```

In the second terminal do anything with these credential  
API endpoint = 127.0.0.1:9000  
Access key = 'minioadmin'  
Access secret = 'minioadmin'  